#Mounting drive

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Installing Librosa


In [ ]:
!pip install librosa soundfile numpy sklearn pyaudio

  ERROR: Failed building wheel for pyaudio
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-5chrgmv1/pyaudio/setup.py'"'"'; __file__='"'"'/tmp/pip-install-5chrgmv1/pyaudio/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-iisb3sln/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [ ]:
!pip install soundfile

#Importing Libraries

In [ ]:
from zipfile import ZipFile 
import librosa
import soundfile
import os, glob, pickle
import numpy as np
import sklearn
sklearn.__version__
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


#Reading Dataset

In [ ]:
file_name = "/content/drive/MyDrive/Speech_Recognition_Project/speech_recogniitiion_dataset.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

#Extract features (mfcc, chroma, mel) from a sound file

In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file

def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(os.path.join(file_name),res_type = 'kaiser_fast')
    if chroma:
         stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))  
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result


       
          
       

# Emotions in the RAVDESS dataset

In [ ]:

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

# Emotions to observe
observed_emotions=[ 'calm', 'happy','fearful', 'disgust','surprised','sad','neutral','angry' ]

# Load the data and extract features for each sound file

In [ ]:

def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Speech_Recognition_Project/speech_recogniton/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size,train_size = 0.75,random_state=9)


# Passing sample audio files

In [ ]:
file = "/content/drive/MyDrive/Speech_Recognition_Project/speech_recogniton/Actor_08/03-01-01-01-01-02-08.wav"
feature = extract_feature(file,mfcc = True,chroma = True, mel = True)

#Split the dataset

In [ ]:

import time
x_train,x_test,y_train,y_test=load_data(test_size = 0.25)
print(x_train,x_test,y_train,y_test)

[[-6.48059387e+02  4.14927063e+01 -5.28695917e+00 ...  5.04699938e-06
   1.29547402e-06  2.20142169e-07]
 [-6.12666992e+02  5.87811394e+01  6.70590734e+00 ...  1.64297071e-05
   1.77926165e-06  9.44622229e-08]
 [-3.59744324e+02  3.31677666e+01 -4.15778770e+01 ...  8.25019160e-05
   1.54266036e-05  1.31189654e-06]
 ...
 [-6.56254700e+02  3.80709419e+01 -9.98252392e+00 ...  2.72932994e-06
   4.77005699e-07  3.38207506e-08]
 [-6.30861084e+02  5.12848167e+01  7.66397762e+00 ...  5.94629319e-06
   1.22769825e-06  4.05802041e-08]
 [-6.56698181e+02  7.84913483e+01 -8.27599430e+00 ...  1.06471994e-07
   1.84467250e-08  1.59651037e-09]] [[-5.52448853e+02  6.93694382e+01 -1.46787777e+01 ...  1.96920928e-06
   3.44458527e-07  4.33093632e-08]
 [-5.92841248e+02  7.16551208e+01 -1.03371925e+01 ...  2.40116651e-06
   4.24073448e-07  4.07918819e-08]
 [-6.76255920e+02  5.47155190e+01 -5.20402527e+00 ...  5.02197827e-06
   7.18188574e-07  8.16106294e-08]
 ...
 [-5.87038330e+02  3.50772133e+01 -1.1080049

#Shaping the model



In [ ]:
#get the shape of the training and testing data 
print((x_train.shape[0],x_test.shape[0]))

(456, 153)


#Get the number of features extracted

In [ ]:
#print((x_train[0], x_test[0]))
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


#Initialize the Multi Layer Perceptron Classifier

In [ ]:

model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)


#  fit/Train the model

In [ ]:
#fit 0r train the model
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

# Predict the accuracy of the model

In [ ]:

y_pred=model.predict(x_test)

y_pre = model.predict([feature])
print(y_pre)
time.sleep(2)


['disgust']


# Calculate the accuracy of our model

In [ ]:

accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 75.82%
